# Getting Started with AstroBkgInterp: JWST/MIRI Imaging Example

This notebook presents an analysis of JWST imaging data of the Type II supernova SN 2021afdx (Engesser, M., TNSAN.155, 2022), which was imaged serendipitously as part of the Early Release Observations of the Cartwheel Galaxy [Program 2727](https://www.stsci.edu/jwst/science-execution/program-information?id=2727) (PI: Pontoppidan). This tutorial offers a detailed, step-by-step guide for estimating and subtracting backgrounds using the AstroBkgInterp package. By the end of this notebook, you will gain hands-on experience with the tools and techniques provided by AstroBkgInterp. This knowledge will enable you to apply the package to similar datasets, whether they are 2D or 3D, or obtained from other instruments or telescopes.

## Contents
1. [Introduction](#intro)<br>
    1.1. [Package Imports](#imports)<br>
    1.2. [Setup](#setup)<br>
2. [Source-Masking](#source-masking)<br>
    2.1. [Locate the source using `Photutils`](#loc)<br>
    2.2. [Set the aperture and annulus sizes](#set) <br>
    2.3. [Verify the source-masking](#verify) <br>
3. [Background Modelling](#bkg) <br>
    3.1. [Estimate and subtract the background](#run) <br>


<a id="intro"></a>
## Introduction 
Core-collapse supernovae (CCSNe) are critical probes of massive stellar evolution and chemical enrichment in galaxies. However, they are often embedded in crowded or high surface-brightness regions of their host galaxies, where background emission is spatially structured and varies on small angular scales. In such environments, precise background estimation is essential for isolating the supernova signal and deriving reliable photometry. 

This notebook focuses on the Type II Supernova SN 2021afdx, first identified in JWST observations of the Cartwheel Galaxy by Engesser et al. (2022). The supernova (SN) is superimposed on a bright, inctricately structured mid-infrared background, comprising of both telescope/thermal contributions and astrophysical emission from the host galaxy. Accurately subtracting this background is therefore essential to recover the intrinsic SN flux.

To address this challenge, we use `AstroBkgInterp` to perform accurate, spatially resolved background subtraction tailored to the complex environment of SN 2021afdx. First, we apply its source-masking procedure to mask the SN and interpolate over its location, providing an estimate of the underlying background and ensuring that the SN flux does not bias the final fit. We then use `AstroBkgInterp` to construct a global background model from the masked data, which is subsequently subtracted from the original image to isolate the SN signal.

<a id="imports"></a>
### Import Packages

- `AstroBkgInterp` is our background estimation tool
- `numpy` for array processing and math
- `atropy.io` for accessing the data
- `astropy.time` for timing
- `astropy.stats` for calculating statistics on the data
- `matplotlib` for plotting images and spectra
- `photutils.detection` for finding sources in the data

In [ ]:
import copy
import os

# Import the background subtraction tool 
from AstroBkgInterp.AstroBkgInterp import AstroBkgInterp

# Import astropy packages
from astropy.io import fits
from astropy.table import Table
from astropy.stats import sigma_clipped_stats
import astropy.units as u

# Import packages for displaying images in notebook
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
from mpl_toolkits.axes_grid1 import make_axes_locatable

# For data handling
import numpy as np

# To find stars in the MRS spectralcubes 
from photutils.detection import DAOStarFinder

<a id="setup"></a>
### Setup

#### Set path to Data
For this notebook, we will analyze a fully calibrated 2D JWST/MIRI image taken with the F770W filter.

In [ ]:
data_url = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_MRS_bkg_notebook/jw02727-o007_t062_miri_f770w_i2d.fits" 

#### Open and display the data 

In [ ]:
hdu = fits.open(data_url)
data = hdu[1].data
err = hdu[2].data

# set all NaN values to 0
data[np.isnan(data)] = 0

plt.imshow(data[400:1600,700:1500], vmax=8, vmin=5.5, origin='lower')
plt.colorbar()


Here we will take a cutout of the full image containing our target SN 2021afdx. 

In [ ]:
data_cutout = data[595:655,1369:1429]
err_cutout = err[595:655,1369:1429]

plt.imshow(data_cutout, vmax=7.5, vmin=6.0, origin='lower')

<a id="source-masking"></a>
## Source-Masking Procedure

A critical feature of `AstroBkgInterp` is its ability to exclude flux from astrophysical sources during the background estimation process. It implements a flexible and configurable source-masking routine that enables users to define regions of interest (typically containing one or more sources) to be excluded from the final background model. 

The user defines the position of the target source, along with an aperture geometry and size; both circular and elliptical apertures are supported. These apertures define the region to be masked and replaced with an interpolated estimate of the underlying background. In polar coordinates centered on the source, the algorithm computes the median flux at each angle from the surrounding annulus and interpolates across the masked region. This directionally informed approach preserves local background structure while excluding contamination from the source itself. To increase robustness and minimize interpolation artifacts, the procedure is repeated in a grid of eight dithered positions surrounding the central source, and the resulting estimates are median-combined to produce the final background.

For 3D data cubes, `AstroBkgInterp` supports wavelength-dependent masking, allowing for the mask to vary as a function of wavelength. This capability accommodates the variation in PSF size with wavelength, and spectral variations in source morphology, ensuring accurate background estimation across the full spectral range.

<a id="loc"></a>
### 1) Locate the source using `Photutils` 

To utilize `AstroBkgInterp`'s source-masking algorithm, we first need to identify the position of the source. Rather than relying solely on the coordinates provided in the image header, we begin by identifying the brightest source in the data cube, which we assume to be the supernova (SN). To accomplish this, we use `DAOStarFinder`, an implementation of the DAOFIND algorithm [Stetson 1987](https://ui.adsabs.harvard.edu/abs/1987PASP...99..191S/abstract) designed for point-source detection in astronomical images. DAOFIND locates local maxima in the image that exceed a specified `threshold` (applied to a convolved image) and have a size and shape similar to the defined 2D Gaussian kernel. 

**Note**: both the detection threshold and maximum separation allowed between neighboring sources should be adjusted based on the specific science case (e.g., the number of stars in the field, the degree of crowding, and the expected brightness distribution).

In [ ]:
mean, median, std = sigma_clipped_stats(data_cutout, sigma=3.0)

# Get a list of sources using a dedicated source detection algorithm
# Find sources at least 3* background (typically)

daofind = DAOStarFinder(fwhm=3.0, threshold=3.*std)
sources = daofind(data_cutout-median) 
print("\n Number of sources in field:", len(sources))

# Positions in pixels
positions = Table([sources['xcentroid'], sources['ycentroid']])

# Convert to RA & Dec (ICRS)
peakpixval = np.zeros(len(sources['xcentroid']))

for count_s, _ in enumerate(sources):
    peakpixval[count_s] = data_cutout[int(np.round(sources['xcentroid'][count_s])), int(np.round(sources['ycentroid'][count_s]))]

# Set the peak pixel positions as the source position. 
src_x, src_y = sources['xcentroid'][np.argmax(peakpixval)], sources['ycentroid'][np.argmax(peakpixval)]
print(f'peak pixel x = {src_x}')
print(f'peak pixel y = {src_y}')

In [ ]:
# Plot all of the sources
plt.imshow(data_cutout, vmax=7.5, vmin=6.0, origin='lower')
plt.colorbar()
plt.scatter(sources['xcentroid'], sources['ycentroid'], c="red", marker="+", s=50)
plt.scatter(src_x, src_y, c="black", marker="+", s=50)

<a id="set"></a>
### 2) Set the aperture and annulus sizes

The next step is to define the aperture and annulus radii for the source-masking procedure. The aperture defines the region to be masked (i.e., the area occupied by the source), while the annulus provides the surrounding region from which the background underneath the source is estimated.

When selecting an aperture radius, the goal is to encompass as much of the source flux as possible, ensuring that the annulus lies beyond the majority of the source light. At the same time, the annulus should remain close enough to the source that the sampled background is representative of the local environment. The annulus must also be sufficiently wide to include enough pixels for a reliable statistical estimate of the underlying background.

In [ ]:
aper_rad = 7
ann_width = 4

plt.figure(figsize=(5, 5))
plt.imshow(data_cutout, vmax=8, vmin=6, origin='lower')
plt.colorbar()

circ = Circle((src_x, src_y), radius=aper_rad, color='r', fill=False)
annin = Circle((src_x, src_y), radius=aper_rad+ann_width, color='r', fill=False)
plt.gca().add_patch(circ)
plt.gca().add_patch(annin)

<a id="verify"></a>
### 3) Verify the source-masking 

To verify the source masking and manually refine the aperture and annulus radii if needed, `AstroBkgInterp` can be run with `bkg_mode = None`. In this mode, the source-masking routine executes as usual, but the tool does not construct a spatial background model across the entire frame. The result is effectively a "source replacement" only, where the background is interpolated locally beneath each masked region without estimating or subtracting a broader background across the image.

In [ ]:
bi = AstroBkgInterp()

# Source position
bi.src_y = src_y
bi.src_x = src_x

# Source masking params
bi.aper_rad = aper_rad
bi.ann_width = ann_width

# Background params
bi.bkg_mode = 'None' 

# Multiprocessing params
bi.pool_size = 12 

bi.k = 2
#uncertainties = False

#bi.is_cube = False

diff, bkg, mask, sigma = bi.run(data_cutout)

Inspect the source masked data.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(6, 5), sharex=True)

ax[0].set_title('Raw Data')
ax[1].set_title('Source Masked Data')

a = ax[0].imshow(data_cutout, vmax=7.5, vmin=6.0, origin='lower', cmap='viridis')
b = ax[1].imshow(bkg, vmax=7.5, vmin=6.0, origin='lower', cmap='viridis')

divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(b, cax=cax, orientation='vertical')

plt.tight_layout()
plt.show()

Once we are happy with the source masking, we can move on to the background modelling stage.

<a id="bkg"></a>
## Background Modelling

In addition to localized source masking, `AstroBkgInterp` provides robust background modelling capabilities designed to handle spatially complex astrophysical sources. After masking flux in the user-defined source regions, the tool constructs a background model using one of several configurable fitting methods. These include a fast, one-dimensional row/column median estimator (`bkg_mode = simple`) and a more sophisticated two-dimensional polynomial surface fitting approach (`bkg_mode = polynomial`). The background model is generated from the source-masked image and is applied on a per-frame basis for 2D images, and on a per-slice basis for 3D data cubes. 

<a id="run"></a>
### 4) Estimate and subtract the background

For the purpose of this notebook, we will set the `"polynomial"` background mode. This mode constructs a background model using a 2D polynomial surface fitting approach implemented in a sliding-window framework. The image is subdivided into a series of overlapping subregions, and a low-order polynomial is independently fit to each region. These fits are then median combined at each pixel location, effectively mitigating the influence of outliers, cosmic rays, and localized noise features. The result is a smooth and robust background model that preserves large-scale background structure while expressing small-scale fluctuations. 

We set the following parameters to control the behavior and resolution of the polynomial fitting process:

- `k`: Specifies the degree of the 2D polynomial used for each local fit. Higher values allow for greater flexibility in modeling complex background structure but may increase the risk of overfitting. We adopt the default value of `k = 3`, which provides a good balance between smoothness and adaptability.

- `bin_size`: Defines the size of the subregions (windows) over which the local polynomial fitting is performed. Smaller windows capture finer spatial variations but increase computational cost, while larger windows produce smoother fits at the expense of resolving small-scale structure. For this work, we use a bin size of `5`.

- `cube_resolution`: Controls the spatial sampling and density of polynomial fits across the image. It determines both the number of local fits and the step size between successive fitting regions. We set `cube_resolution = high`, which enforces the finest sampling, yielding a smoother and more detailed background model at the cost of increased computation time.

In [ ]:
bi = AstroBkgInterp()

# Source position
bi.src_y = src_y
bi.src_x = src_x

# Source masking params
bi.aper_rad = aper_rad
bi.ann_width = ann_width

# Background params
bi.bkg_mode = 'polynomial' 
bi.bin_size = 9 
bi.cube_resolution = 'high' 

# Multiprocessing params
bi.pool_size = 12 
uncertainties = False

#diff, bkg, mask, diff_err = bi.run(data_cutout)
diff, bkg, mask, sigma = bi.run(data_cutout, err=err_cutout)

Save out new background subtracted data.

In [ ]:
newdata = np.array([s for s in diff])
newhdu = copy.deepcopy(hdu)
newhdu[1].data = newdata
newhdu.writeto('newdata_high_res.fits', overwrite=True)

Now lets plot out all the products returned by `AstroBkgInterp`.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 12))

ax[0, 0].set_title('Raw Data')
ax[0, 1].set_title('Source-Masked data')
ax[1, 0].set_title('Background Fit')
ax[1, 1].set_title('Residual')

a = ax[0, 0].imshow(data_cutout, vmax=7.5, vmin=6.0, origin='lower')
b = ax[0, 1].imshow(mask, vmax=7.5, vmin=6.0, origin='lower')
c = ax[1, 0].imshow(bkg, vmax=7.5, vmin=6.0, origin='lower')
d = ax[1, 1].imshow(diff, origin='lower')

divider = make_axes_locatable(ax[1, 1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(d, cax=cax, orientation='vertical')

divider = make_axes_locatable(ax[1, 1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(d, cax=cax, orientation='vertical')

divider = make_axes_locatable(ax[1, 1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(d, cax=cax, orientation='vertical')

divider = make_axes_locatable(ax[1, 1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(d, cax=cax, orientation='vertical')

plt.tight_layout()

plt.show()

And take a look at the uncertainty

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(10,10),sharex=True,sharey=True)

ax[0,0].set_title(r'Data',fontsize=15)
ax[0,1].set_title('Difference',fontsize=15)
ax[1,0].set_title(r'Error ($\sigma$)',fontsize=15)
ax[1,1].set_title(r'$\sigma$ Final',fontsize=15)

im1 = ax[0,0].imshow(data_cutout, vmin=6, vmax=7.5, origin='lower')
im2 = ax[0,1].imshow(diff, origin='lower')
im3 = ax[1,0].imshow(err_cutout, vmin=0, vmax=0.05, origin='lower')
im4 = ax[1,1].imshow(sigma, vmin=0, vmax=0.05, origin='lower')

divider = make_axes_locatable(ax[0,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im1, cax=cax, orientation='vertical')

divider = make_axes_locatable(ax[0,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im2, cax=cax, orientation='vertical')

divider = make_axes_locatable(ax[1,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im3, cax=cax, orientation='vertical')

divider = make_axes_locatable(ax[1,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im4, cax=cax, orientation='vertical')

plt.tight_layout()
#plt.savefig('uncertainty_prop')
plt.show()